In [33]:
from IPython.display import display
import os

# 上傳檔案（會跳出檔案選擇器）
from ipywidgets import FileUpload

upload = FileUpload()
display(upload)

FileUpload(value={}, description='Upload')

In [34]:
import os
from pathlib import Path

# 假設你只上傳了一個檔案
# Check if any file is uploaded and get the filename
if upload.value:
    # Get the first (and likely only) filename from the dictionary keys
    filename = list(upload.value.keys())[0]
    fileinfo = upload.value[filename] # Access the file info dictionary using the filename as key
else:
    print("No file uploaded.")
    # Handle the case where no file is uploaded, perhaps by exiting or prompting the user.
    # For this example, we'll assume a file was uploaded as per the traceback context.
    raise FileNotFoundError("No file was uploaded.")


# 顯示內容結構（除錯用）
print(fileinfo)

# 儲存 kaggle.json
# filename is already obtained above
content = fileinfo['content']

kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(exist_ok=True)

kaggle_json_path = kaggle_dir / "kaggle.json"
with open(kaggle_json_path, "wb") as f:
    f.write(content)

# 設定權限（Linux/macOS 建議）
os.chmod(kaggle_json_path, 0o600)

print(f"{filename} 已成功儲存至 {kaggle_json_path}")

{'metadata': {'name': 'kaggle.json', 'type': 'application/json', 'size': 64, 'lastModified': 1745215961986}, 'content': b'{"username":"suchiwen","key":"01a925cee9e9e9d232008524b0434fb9"}'}
kaggle.json 已成功儲存至 /root/.kaggle/kaggle.json


In [35]:
!kaggle datasets list -s cifar


ref                                                     title                                                  size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------  ----------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
fedesoriano/cifar100                                    CIFAR-100 Python                                  168517809  2020-12-26 08:37:10.143000          12526        178  1.0              
pankrzysiu/cifar10-python                               CIFAR-10 Python                                   340613496  2018-01-27 13:42:40.967000          15066        255  0.75             
petitbonney/cifar10-image-recognition                   CIFAR-10                                         1007971063  2019-10-01 12:50:23.227000           2977         27  0.8235294        
valentynsichkar/cifar10-preprocessed                   

In [36]:
!pip install -U kaggle
!pip install --upgrade pandas
import os
import zipfile

# 建立 Kaggle 資料夾
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 下載 Dog Breed Identification 資料集
!kaggle competitions download -c dog-breed-identification --force
!unzip -oq dog-breed-identification.zip -d dog-breed-identification


import pandas as pd
import numpy as np

labels = pd.read_csv('dog-breed-identification/labels.csv')


cp: cannot stat 'kaggle.json': No such file or directory
 96% 662M/691M [00:05<00:00, 42.2MB/s]
100% 691M/691M [00:05<00:00, 132MB/s] 


In [48]:
import os
import pandas as pd
from PIL import Image
from collections import Counter
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, random_split
from torchvision import transforms

# ---------- 參數設定 ----------
data_dir = "dog-breed-identification"
train_dir = os.path.join(data_dir, "train")
labels_path = os.path.join(data_dir, "labels.csv")
val_ratio = 0.2
batch_size = 32
num_workers = 2
gamma = 2.0  # Focal Loss gamma
num_epochs = 10
lr = 1e-4

# ---------- 資料增強 ----------
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

def cutmix(data, targets, alpha=1.0):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    data[:, :, bbx1:bbx2, bby1:bby2] = shuffled_data[:, :, bbx1:bbx2, bby1:bby2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size(-1) * data.size(-2)))
    return data, targets, shuffled_targets, lam

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.1, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    RandomErasing(p=0.5),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])



In [49]:
# ---------- 自訂 Dataset ----------
class DogBreedDataset(Dataset):
    def __init__(self, image_dir, labels_df, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.labels_df = labels_df
        self.breed2idx = {breed: idx for idx, breed in enumerate(sorted(labels_df['breed'].unique()))}
        self.labels_df['label'] = self.labels_df['breed'].map(self.breed2idx)

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        row = self.labels_df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['id'] + ".jpg")
        image = Image.open(img_path).convert("RGB")
        label = row['label']
        if self.transform:
            image = self.transform(image)
        return image, label

# ---------- 讀入標籤 ----------
labels = pd.read_csv(labels_path)

# ---------- 建立 Dataset 與切分 ----------
full_dataset = DogBreedDataset(train_dir, labels, transform=train_transform)
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 替驗證集設定不同 transform
val_dataset.dataset.transform = val_transform

# ---------- Weighted Sampler ----------
train_labels = [full_dataset.labels_df.iloc[i]['label'] for i in train_dataset.indices]
label_counts = Counter(train_labels)
class_sample_counts = [label_counts[i] for i in range(len(label_counts))]
sample_weights = [1.0 / class_sample_counts[label] for label in train_labels]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# ---------- Dataloader ----------
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# ---------- Focal Loss ----------
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, input, target):
        logpt = -self.ce(input, target)
        pt = torch.exp(logpt)
        focal_loss = -((1 - pt) ** self.gamma) * logpt
        return focal_loss.mean()

criterion = FocalLoss(gamma=gamma)

print("✅ 資料載入與 FocalLoss 建立成功！")


✅ 資料載入與 FocalLoss 建立成功！


In [50]:
# ---------- 模型 ----------
model = models.vgg16(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False
model.classifier[6] = nn.Linear(model.classifier[6].in_features, len(label_counts))
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
scaler = GradScaler()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-50-e2520c57adca>:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [52]:
# ---------- 訓練 ----------
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        # CutMix 機率
        if np.random.rand() < 0.5:
            imgs, targets1, targets2, lam = cutmix(imgs, labels)
            outputs = model(imgs)
            loss = lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)
        else:
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    # ---------- 驗證 ----------
    model.eval()
    val_loss, correct, total = 0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total
    scheduler.step()

    print(f"[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}")


[Epoch 1] Train Loss: 13049.5284 | Val Loss: 7.1482 | Val Acc: 0.0117
[Epoch 2] Train Loss: 421.9135 | Val Loss: 6.9582 | Val Acc: 0.0117
[Epoch 3] Train Loss: 174.1341 | Val Loss: 17.0666 | Val Acc: 0.0108
[Epoch 4] Train Loss: 90.1080 | Val Loss: 8.2442 | Val Acc: 0.0098
[Epoch 5] Train Loss: 72.8263 | Val Loss: 5.9354 | Val Acc: 0.0108
[Epoch 6] Train Loss: 98.2779 | Val Loss: 6.1145 | Val Acc: 0.0083
[Epoch 7] Train Loss: 81.7987 | Val Loss: 5.2492 | Val Acc: 0.0108
[Epoch 8] Train Loss: 44.6131 | Val Loss: 4.9535 | Val Acc: 0.0064
[Epoch 9] Train Loss: 46.3532 | Val Loss: 4.8883 | Val Acc: 0.0147
[Epoch 10] Train Loss: 48.6222 | Val Loss: 4.9231 | Val Acc: 0.0073


In [57]:
# ---------- 預測 ----------
model.eval()
all_probs = []
all_img_ids = []

with torch.no_grad():
    for images, img_ids in test_loader:
        images = images.to(device)
        outputs = model(images)  # logits
        probs = torch.softmax(outputs, dim=1)  # 機率分布

        all_probs.extend(probs.cpu().numpy())
        all_img_ids.extend([img_id.replace(".jpg", "") for img_id in img_ids])

# ---------- 產生 submission.csv ----------
# 取得所有類別（按排序）
class_names = sorted(labels_df['breed'].unique())

# 組成 DataFrame
submission = pd.DataFrame(all_probs, columns=class_names)
submission.insert(0, "id", all_img_ids)

# 儲存成 CSV
submission.to_csv("submission.csv", index=False)
print("✅ 已成功產生符合 Kaggle 標準的 submission.csv！")


✅ 已成功產生符合 Kaggle 標準的 submission.csv！
